In [1]:
! pip install azure-storage-blob requests pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 28.5 MB/s eta 0:00:0000:01


In [21]:
import requests as req
import pandas as pd
from azure.storage.blob import BlobServiceClient
import os
from tqdm import tqdm
import json
from datetime import datetime, timedelta


In [ ]:
os.environ["AZURE_CONNECTION_STRING"] = "key"
os.environ["RAPID_API_KEY"] = "key"
os.environ["RAPID_API_HOST"] = "host"
os.environ["OPENROUTE_KEY"] = 'key'

In [3]:
connection_string = os.getenv('AZURE_CONNECTION_STRING')
blob_service_client = BlobServiceClient.from_connection_string(connection_string)

In [4]:
def upload_blob(client, blob_name, local_file_path):
  container_name = "bdmcontainerp1"
  container_client = blob_service_client.get_container_client(container_name)

  with open(local_file_path, "rb") as data:
      container_client.upload_blob(name=blob_name, data=data, overwrite=True)

  blobs = [blob.name for blob in container_client.list_blobs()]
  if blob_name not in blobs:
    raise Exception("Blob was not uploaded")

# Weather Data
Check for more examples:  https://open-meteo.com/


In [ ]:
latitude = 41.3874
longitude = 2.1686
city_coords = {
                'Barcelona': {'latitude': 41.3874, 'longitude': 2.1686},
                'Paris': {'latitude': 48.8575, 'longitude': 2.3514},
                'Madrid': {'latitude': 40.4167, 'longitude': 3.7033},
                'Rome': {'latitude': 41.8967, 'longitude': 12.4822}
                }

start_date = '2025-03-30'
end_date = '2025-03-30'
hourly = 'temperature_2m,precipitation_probability,apparent_temperature,precipitation,cloud_cover,wind_speed_10m'
querystring = {
                'latitude':latitude,
                'longitude':longitude,
                'hourly':hourly,
                'start_date':start_date,
                'end_date':end_date
                }

weather_endpoint = f'https://api.open-meteo.com/v1/forecast'
weather_res = req.get(weather_endpoint, params=querystring).json()
weather_res

200

# Accomodation

Check for more examples: https://rapidapi.com/DataCrawler/api/booking-com15/playground/apiendpoint_818c2744-8507-4071-829e-d080b667a06c

In [27]:
def save_images(res, client, querystring, city):
    hotels = res['data']['hotels']
    for hotel in hotels:
        photos = hotel['property']['photoUrls']
        for photo in photos:
            img_res = req.get(photo, stream=True)
            filename = city + '/' + str(hotel['hotel_id']) + '_' + querystring['dest_id'] + '_' + querystring['arrival_date'] + '_' + querystring['departure_date'] + '.jpg'
            if img_res.status_code == 200:
                with open('accomodation_images/' + filename, "wb") as file:
                    for chunk in img_res.iter_content(1024):  # Guardar en bloques de 1024 bytes
                        file.write(chunk)
            upload_blob(client, f'accomodation_images/{filename}', f'accomodation_images/{filename}')
            os.remove(f'accomodation_images/{filename}')

In [28]:
def save_json(res, filename):
    with open(filename, 'w') as file:
        json.dump(res, file)

In [ ]:
accomodation_endpoint = "https://booking-com15.p.rapidapi.com/api/v1/hotels/searchHotels"
weather_endpoint = f'https://api.open-meteo.com/v1/forecast'

delta = timedelta(days=1)

destination_ids = {
    "Barcelona": "-372490",
    "Rome": "-126693",
    "Madrid": "-390625",
    "Paris": "-1456928"
}

headers = {
            "x-rapidapi-key": os.environ["RAPID_API_KEY"],
            "x-rapidapi-host": os.environ["RAPID_API_HOST"]
        }

accomodation_querystring = {
            "dest_id": '',
            "search_type": "CITY",
            "arrival_date": '',
            "departure_date": '',
            "adults": "2",
            "children_age": "0",
            "room_qty": "1",
            "page_number": "1",
            "units": "metric",
            "temperature_unit": "c",
            "languagecode": "en-us",
            "currency_code": "EUR"
        }

destination_coords = {
                'Barcelona': {'latitude': 41.3874, 'longitude': 2.1686},
                'Paris': {'latitude': 48.8575, 'longitude': 2.3514},
                'Madrid': {'latitude': 40.4167, 'longitude': 3.7033},
                'Rome': {'latitude': 41.8967, 'longitude': 12.4822}
                }

weather_metrics = 'temperature_2m,precipitation_probability,apparent_temperature,precipitation,cloud_cover,wind_speed_10m'
weather_querystring = {
                'latitude': '',
                'longitude': '',
                'hourly': weather_metrics,
                'start_date': '',
                'end_date': ''
                }

start = datetime.strptime('2025-03-24', "%Y-%m-%d")
end = datetime.strptime('2025-03-25', "%Y-%m-%d")
try:
    for single_date in tqdm([start + i * delta for i in range((end - start).days + 1)]):
        arrival_date = single_date.strftime("%Y-%m-%d")
        departure_date = (single_date + timedelta(days=1)).strftime("%Y-%m-%d")
        start_date = single_date.replace(year=single_date.year - 1).strftime("%Y-%m-%d")
        end_date = (single_date + timedelta(days=1)).replace(year=single_date.year - 1).strftime("%Y-%m-%d")

        for city, city_id in destination_ids.items():
            
            accomodation_querystring['dest_id'] = city_id
            accomodation_querystring['arrival_date'] = arrival_date
            accomodation_querystring['departure_date'] = departure_date
            
            accomodation_res = req.get(accomodation_endpoint, headers=headers, params=accomodation_querystring)
            if accomodation_res.status_code == 200:
                accomodation_res = accomodation_res.json()
                save_images(accomodation_res, blob_service_client, accomodation_querystring, city)
                json_res_filename =  city + '/' + accomodation_querystring['arrival_date'] + '_' + accomodation_querystring['departure_date'] + '.json'
                save_json(accomodation_res, f'accomodation/{json_res_filename}')
                upload_blob(blob_service_client, f'accomodation/{json_res_filename}', f'accomodation/{json_res_filename}')
                os.remove(f'accomodation/{json_res_filename}')
            else:
                print(city, arrival_date, departure_date)

            latitude, longitude = destination_coords[city].values()
            weather_querystring['latitude'] = latitude
            weather_querystring['longitude'] = longitude
            weather_querystring['start_date'] = start_date
            weather_querystring['end_date'] = start_date
            weather_json_filename = city + '/' + start_date + '.json'
            
            weather_res = req.get(weather_endpoint, params=weather_querystring)
            if weather_res.status_code == 200:
                weather_res = weather_res.json()
                save_json(weather_res, f'weather/{weather_json_filename}')
                upload_blob(blob_service_client, f'weather/{weather_json_filename}', f'weather/{weather_json_filename}')
                os.remove(f'weather/{weather_json_filename}')
            else:
                print(city, start_date, end_date)
except Exception as e:
    print(e)
    print(city, arrival_date, departure_date)

 50%|█████     | 1/2 [01:45<01:45, 105.33s/it]

# Transportation

Find out more in: https://openrouteservice.org/dev/#/api-docs/optimization/post
Alternatives: https://github.com/graphhopper/graphhopper/blob/master/README.md#Map-Matching
https://github.com/VROOM-Project/vroom/blob/master/docs/API.md